# Género y tecnología: ¿qué eligen los estudiantes?

In [144]:
from collections import namedtuple

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from scipy import stats
import seaborn as sns
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact
import folium
from folium.plugins import DualMap 
from datetime import datetime
from IPython.display import HTML, display

# módulos propios
from scripts import mapas, pygal_helpers, georef, download
from scripts.pygal_helpers import df_to_pygal_line, get_pygal_html
from scripts.helpers import read_csv, format_pct, get_bins

matplotlib.style.use("ggplot")

## Evolución de género en grupo de carreras

In [145]:
df_evolucion_carreras = read_csv('grupos_evolucion_genero.csv')
df_evolucion_grupos_carreras = read_csv('grupos_carreras_evolucion_genero.csv')

df_evolucion_carreras.head(10)

,anio,carrera_grupo,estudiantes_total,estudiantes_varones,estudiantes_mujeres,nuevos_inscriptos_total,nuevos_inscriptos_varones,nuevos_inscriptos_mujeres,egresados_total,egresados_varones,egresados_mujeres,estudiantes_mujeres_pct,inscripciones_mujeres_pct,estudiantes_hombres_pct
0,2011,Arquitectura y Urbanismo,60303.0,31217.0,29086.0,9743.0,4859.0,4884.0,3064.0,1412.0,1652.0,0.482331,0.501283,0.517669
1,2011,Artes,53170.0,22164.0,31006.0,13305.0,6016.0,7289.0,1418.0,503.0,915.0,0.583148,0.547839,0.416852
2,2011,Comunicacion,113474.0,40764.0,72710.0,25473.0,9914.0,15559.0,7104.0,2141.0,4963.0,0.640764,0.610804,0.359236
3,2011,Diseño,52552.0,18574.0,33978.0,11127.0,3813.0,7314.0,2826.0,820.0,2006.0,0.646560,0.657320,0.353440
4,2011,Economicas,318405.0,149696.0,168709.0,67062.0,33017.0,34045.0,19059.0,8970.0,10089.0,0.529857,0.507665,0.470143
5,2011,Educacion,139802.0,32933.0,106869.0,42465.0,10907.0,31558.0,8611.0,1814.0,6797.0,0.764431,0.743153,0.235569
6,2011,Exactas,23899.0,11515.0,12384.0,5491.0,2516.0,2975.0,1479.0,633.0,846.0,0.518181,0.541796,0.481819
7,2011,"Hoteleria, Gastronomia y Turismo",25790.0,8221.0,17569.0,6917.0,2395.0,4522.0,1513.0,364.0,1149.0,0.681233,0.653752,0.318767
8,2011,Informatica,163968.0,132685.0,31283.0,37090.0,30891.0,6199.0,7554.0,5702.0,1852.0,0.190787,0.167134,0.809213
9,2011,Ingenieria,124298.0,101296.0,23002.0,25854.0,20815.0,5039.0,5885.0,4682.0,1203.0,0.185055,0.194902,0.814945


In [146]:
#No usamos el año 2010 para la comparacion ya que la UBA no entregó datos para ese año
df_evolucion_carreras = df_evolucion_carreras[df_evolucion_carreras.anio != 2010]

In [147]:
df_evolucion_grupos_carreras = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.anio !=2010]

### ¿Cómo evolucionó la proporción de estudiantes femeninos en distintas disciplinas?

In [148]:
# gráfico de evolución hombres / mujeres por grupo de carreras

# widget: seleccionar grupo temático de carreras
variables_carreras_evolucion = tuple(df_evolucion_carreras.carrera_grupo.unique())

widget_grupos=widgets.SelectMultiple(
    options=sorted(variables_carreras_evolucion),
    value=["Informatica"],
    description='Areas Tematicas',
    disabled=False
)

# widget: elegir indicador
variables_indicadores_line = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Inscripciones totales" : "nuevos_inscriptos_total",
    "Egresos totales" : "egresados_total"
}
widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores_line.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

@interact(carrera_grupo=widget_grupos, indicador=widget_indicador)
def generate_chart_evolucion_grupos(carrera_grupo, indicador):
    
    serie_grupo_carrera = df_evolucion_carreras[
        df_evolucion_carreras.carrera_grupo.isin(carrera_grupo)][[
            "anio", 'carrera_grupo',  indicador]]
    
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='carrera_grupo', values=indicador)
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(8,), options=('Arquitectura y Urban…

In [149]:
print(tuple(df_evolucion_carreras.carrera_grupo.unique()))


('Arquitectura y Urbanismo', 'Artes', 'Comunicacion', 'Diseño', 'Economicas', 'Educacion', 'Exactas', 'Hoteleria, Gastronomia y Turismo', 'Informatica', 'Ingenieria', 'Juridicas', 'Naturales', 'Otros', 'Psicologia', 'Salud', 'Sociales')


In [150]:
#Grafico con todos los grupos, sin interaccion                   
generate_chart_evolucion_grupos(sorted(variables_carreras_evolucion),"estudiantes_mujeres_pct")

In [151]:
# gráfico de evolución de las carreras dentro de un grupo

@interact(carrera_grupo=widget_grupos) 
def generate_chart_evolucion_titulos(carrera_grupo):
    
    serie_grupo_carrera = df_evolucion_grupos_carreras[
        df_evolucion_grupos_carreras.carrera_grupo.isin(carrera_grupo)][[
        "anio", 'Carrera_normalizada',  "estudiantes_mujeres_pct"]]
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='Carrera_normalizada', values='estudiantes_mujeres_pct')
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(8,), options=('Arquitectura y Urban…

## Comparación de género por provincias y universidades

In [152]:
df_universidades = pd.read_csv('data/output/universidades_mapa.csv')
df_evolucion_ubicacion = pd.read_csv('data/output/ubicacion_evolucion_genero.csv', 
                                     dtype={'provincia_id': str})
df_evolucion_ubicacion['provincia_id'] = df_evolucion_ubicacion.provincia_id.str.zfill(2)

In [153]:
argentina_coords = [-40, -64]
osm_no_labels = "https://tiles.wmflabs.org/osm-no-labels/{z}/{x}/{y}.png"
ign_labels = "http://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"

## Indicadores por provincia

In [154]:
df_evolucion_ubicacion.head(10)

,anio,provincia_id,carrera_grupo,level_0,index,estudiantes_total,estudiantes_varones,estudiantes_mujeres,nuevos_inscriptos_total,nuevos_inscriptos_varones,nuevos_inscriptos_mujeres,egresados_total,egresados_varones,egresados_mujeres,estudiantes_mujeres_pct,inscripciones_mujeres_pct,estudiantes_hombres_pct,estudiantes_grupo_pct
0,2011,02,Arquitectura y Urbanismo,166256,166256,22156.0,11274.0,10882.0,3697.0,1793.0,1904.0,1304.0,633.0,671.0,0.491154,14.906073,13.355206,0.031802
1,2011,02,Artes,714222,714222,28116.0,9760.0,18356.0,6537.0,2453.0,4084.0,810.0,270.0,540.0,0.652867,42.558890,36.603461,0.040357
2,2011,02,Comunicacion,1227479,1227479,50809.0,18719.0,32090.0,10951.0,4242.0,6709.0,3811.0,1174.0,2637.0,0.631581,88.484257,66.609315,0.072930
3,2011,02,Diseño,332543,332543,30984.0,9342.0,21642.0,6682.0,1995.0,4687.0,1740.0,438.0,1302.0,0.698490,24.346558,16.267825,0.044474
4,2011,02,Economicas,1686778,1686778,111434.0,57606.0,53828.0,22429.0,11747.0,10682.0,8693.0,4439.0,4254.0,0.483048,100.115797,151.333689,0.159951
5,2011,02,Educacion,1055072,1055072,25724.0,5722.0,20002.0,9197.0,2222.0,6975.0,3310.0,786.0,2524.0,0.777562,96.691373,49.989839,0.036924
6,2011,02,Exactas,236546,236546,7378.0,4208.0,3170.0,1360.0,716.0,644.0,336.0,176.0,160.0,0.429656,12.055946,18.032856,0.010590
7,2011,02,"Hoteleria, Gastronomia y Turismo",316319,316319,8024.0,2557.0,5467.0,2481.0,899.0,1582.0,637.0,148.0,489.0,0.681331,26.793734,15.658926,0.011518
8,2011,02,Informatica,837309,837309,53077.0,44424.0,8653.0,11432.0,9688.0,1744.0,3070.0,2278.0,792.0,0.163027,15.181583,101.242114,0.076186
9,2011,02,Ingenieria,610626,610626,31292.0,25351.0,5941.0,6590.0,5282.0,1308.0,1647.0,1226.0,421.0,0.189857,22.235730,96.130763,0.044916


In [155]:
INDICADORES_HEATMAP = {
    "estudiantes_mujeres_pct" : "Ratio de estudiantes mujeres",
    "estudiantes_grupo_pct" : "Ratio de estudiantes",
    "estudiantes_varones" : "Total de estudiantes hombres",
    #"estudiantes_mujeres" : "Total de estudiantes mujeres", 
    #"inscriptos_mujeres_pct" : "Ratio de inscriptas mujeres ",
    #"inscriptos_hombres_pct" : "Ratio de inscriptas hombres ",
    #"estudiantes_hombres_pct" : "Ratio de estudiantes hombres ",
}
variables_indicadores = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio grupo": "estudiantes_grupo_pct",
    "Est. hombres": "estudiantes_varones"
}

variables_carreras_heatmap = sorted(df_evolucion_ubicacion.carrera_grupo.unique())
variables_anio_heatmap = sorted(df_evolucion_ubicacion.anio.unique())

widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

widget_carreras = widgets.Dropdown(
    options=variables_carreras_heatmap,
    description="Grupo : ",
    value="Informatica"
)

widget_anio = widgets.Dropdown(
    options=variables_anio_heatmap,
    description="Año : ",
    value=2016
)

widget_scale = widgets.RadioButtons(
    options=['Cuantiles', 'Fija', 'Relativa'],
    description='Escala :',
    value='Relativa'
)

@interact(anio=widget_anio,
          carrera_grupo=widget_carreras,
          indicador=widget_indicador,
          scale_type=widget_scale
         )
def generate_heat_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa_heatmap = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_heatmap)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa_heatmap)
    
    return mapa_heatmap

interactive(children=(Dropdown(description='Año : ', index=5, options=(2011, 2012, 2013, 2014, 2015, 2016, 201…

## Indicadores por universidad

In [156]:
def mapear_universidades(mapa, df, lat_col, lon_col, institucion_nombre_col, carreras_grupos_col):
    """ cargar informacion al marker de forma masiva.

        Args:
            lat (float): latitud
            lon (float): longuitud
            institucion_nombre (str): nombre de la universidad
            carreras_grupos (list): lista con las areas de estudios ofrecidas

        """
    
    for row in df_universidades.iterrows():
        mapas.crear_marker(
            mapa,
            row[1]["lat"],
            row[1]["lon"],
            row[1]["institucion_nombre"],
            str(row[1]["carrera_grupo"]).split(",")
        )
            

In [157]:
mapa_markers = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_markers)

In [158]:
mapear_universidades(
    mapa_markers, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo"
)

In [159]:
mapa_markers

## Indicadores por universidad Y heatmap

In [160]:
def generate_heat_marker_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa)
    
    mapear_universidades(
    mapa, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo")

    return mapa

In [161]:
generate_heat_marker_map(2016, 'Informatica', 'estudiantes_mujeres_pct')